In [241]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from timeit import default_timer as timer
from scipy.special import softmax



In [242]:
df = pd.read_csv('ESS8_data.csv')

In [269]:
X = df[['SD1', 'PO1', 'UN1', 'AC1', 'SC1',
       'ST1', 'CO1', 'TR1', 'HD1', 'AC2', 'SC2', 'ST2',
       'CO2', 'PO2', 'BE2', 'TR2', 'HD2']].iloc[range(100),:]
X = X.to_numpy().T
N, M = X.T.shape
K = 5

In [270]:
np.random.seed(42)

p = np.random.permutation(range(N))[:K]
Z = X[:,p]
# Z = np.random.randint(1, 6, (M,K)) 

#Q = Z.T @ Z
#Qt = torch.tensor(Q,requires_grad=False).float()

R = X.T @ X
Rt = torch.tensor(R,requires_grad=False).float()

In [271]:
# applies both for eq. 7 and 8
def error(a_i,Qt,qt):
    return (0.5 * a_i.T @ Qt @ a_i) - (qt.T @ a_i)

## Defining the helper functions applyConstraints and furthestSum

applyConstrants ensures that the constraints of the problem are upheld i.e. $e_{ij} \geq 0$ and $1^T \textbf{e}_j = 1$


furthestSum is an effective way to initialize the values of $\textbf{b}_i$

In [272]:
def applyConstrains(M):
    return softmax(M, axis = 0)



#def furthestSum(X):
    #idx = np.random.choice(range(0,N))
    #x_j = X[:,idx]
    #max_vals = [0]
    #max_vals = max_vals * K

    #b_j = np.empty((K,1))
    #b_j_norm = np.empty((K,1))

    #for i in range(N):
    #    val = 0
   #     for j in range(M):
  #          val += np.abs(X[j, i] - x_j[j])
 #       max_idxs = [j for j, v in enumerate(max_vals) if val > v]
#
      #  if len(max_idxs) > 0:
     #       max_vals[max_idxs[0]] = val
    #
    #b_j = b_j.astype(np.float64)
            
    #return b_j


    
# print(b_j)    


[1 1 6 1 1 1 1 6 1 1 1 6 1 1 3 1 3]


In [318]:
def furthestSum(X):
    # Choose a random point for initialization
    idx = int(np.random.choice(range(0,N)))
    x_j = X[:,idx]

    j_news = list()
    j_news.append(idx)

    excluded = [idx]

    # Loop over the K archetypes
    for n in range(K):
        best_val = 0
        best_idx = 0
        # Loop over all unseen samples
        for i in range(N-len(excluded)):
            if i not in excluded:
                val = 0
                # sum over each element for each point
                for ele in j_news:
                    for j in range(M):
                        
                        val += np.abs(X[j,i] - X[j , ele])
                if val > best_val:
                    best_val = val
                    best_idx = i
                    
        # 
        j_news.append(int(best_idx))
        excluded.append(best_idx)
        # Remove the random initialization
        if n == 0:
            j_news.pop(0)
            excluded.pop(0)
        
    return j_news

idxs = furthestSum(X)

init_vals_b = X[init_idxs[i],:].astype(np.float64)
init_vals_bt = torch.tensor(init_vals_b, requires_grad = False).float()

init_vals_b.shape


(100,)

In [315]:
start = timer()
torch.manual_seed(42)

RSS_values = list()
A = np.zeros((K,N))#.tolist()
B = np.zeros((N,K))#.tolist()

# Get initial indexes for B via. furthest sum procedure
init_idxs = furthestSum(X)



# LOOP UNTIL RSS IS LOW
for n in range(2): #N
    Q = Z.T @ Z
    Qt = torch.tensor(Q,requires_grad=False).float()

    # LOOP THROUGH ENTIRE A
    for i in range(N):
        q = Z.T @ X[:,i]
        qt = torch.tensor(q,requires_grad=False).float()
        
        a_i = torch.autograd.Variable(torch.rand(K, 1), requires_grad=True) # eller er det Kx1 ?
        
        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(a_i,Qt,qt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(100): # 100000
            Delta = error(a_i,Qt,qt) 
            L = torch.norm(Delta, p=2)
            L.backward()
            a_i.data -= step_size * a_i.grad.data # step
            a_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for a is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break
        
        A[:,i] = np.array(a_i.tolist()).flatten() 
        
        # print('Loss after: %s' % (torch.norm( error(a_i,Qt,qt) )))

    ### apply softmax here? ###
    
    A = applyConstrains(A)
    Z = X @ A.T @ np.linalg.inv(A@A.T)
    
    
    # LOOP THROUGH ENTIRE B
    for i in range(K): #K
        r = X.T @ Z[:,i]
        rt = torch.tensor(r,requires_grad=False).float()
        
        
        b_i = torch.autograd.Variable(torch.randn(N,1), requires_grad=True)
        
        
        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(b_i,Rt,rt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(b_i,Rt,rt)
            L = torch.norm(Delta, p=2)
            L.backward()
            b_i.data -= step_size * b_i.grad.data # step
            b_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for b is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break

        B[:,i] = np.array(b_i.tolist()).flatten() 
        print('Loss after: %s' % (torch.norm( error(b_i,Rt,rt) )))    
    
    # apply softmax here
    B = applyConstrains(B)
    Z = X @ B
    
    Zt = torch.tensor(Z, requires_grad=False).float()
    At = torch.tensor(A, requires_grad=False).float()
    Xt = torch.tensor(X,requires_grad=False).float()
    print("RSS at n=%s" % n, torch.norm(Xt-Zt@At,p='fro'))
    RSS_values.append( torch.norm(Xt-Zt@At,p='fro'))
    
end = timer()

print("It took: {0} seconds to finish running".format(end - start))
print("The best RSS value was", min(RSS_values))

0
2
10
25
28
83
0
1
2
6
14
1
2
4
12
47
73
1
2
4
12
25
47
Loss before: tensor(295.9209, grad_fn=<NormBackward1>)
Loss for a is tensor(295.9209, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(4.1297, grad_fn=<NormBackward1>)
Loss for a is tensor(4.1297, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(302.6338, grad_fn=<NormBackward1>)
Loss for a is tensor(302.6338, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(197.2050, grad_fn=<NormBackward1>)
Loss for a is tensor(197.2050, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(13.3398, grad_fn=<NormBackward1>)
Loss for a is tensor(13.3398, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(53.1242, grad_fn=<NormBackward1>)
Loss for a is tensor(53.1242, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(79.0898, grad_fn=<NormBackward1>)
Loss for a is tensor(79.0898, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(222.3477, grad_fn=<NormBackward1>)
Loss for a is te

Loss for a is tensor(70.0860, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(65.6009, grad_fn=<NormBackward1>)
Loss for a is tensor(65.6009, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(19.5793, grad_fn=<NormBackward1>)
Loss for a is tensor(19.5793, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(52.4331, grad_fn=<NormBackward1>)
Loss for a is tensor(52.4331, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(57.8545, grad_fn=<NormBackward1>)
Loss for a is tensor(57.8545, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(106.0209, grad_fn=<NormBackward1>)
Loss for a is tensor(106.0209, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(239.7072, grad_fn=<NormBackward1>)
Loss for a is tensor(239.7072, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(54.6731, grad_fn=<NormBackward1>)
Loss for a is tensor(54.6731, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(90.1512, grad_fn=<NormBackward1>)
Loss

Loss for a is tensor(127.2993, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(54.0722, grad_fn=<NormBackward1>)
Loss for a is tensor(54.0722, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(97.1048, grad_fn=<NormBackward1>)
Loss for a is tensor(97.1048, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(183.2425, grad_fn=<NormBackward1>)
Loss for a is tensor(183.2425, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(159.5834, grad_fn=<NormBackward1>)
Loss for a is tensor(159.5834, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(1.5495, grad_fn=<NormBackward1>)
Loss for a is tensor(1.5495, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(71.7983, grad_fn=<NormBackward1>)
Loss for a is tensor(71.7983, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(545.3300, grad_fn=<NormBackward1>)
Loss for a is tensor(545.3300, grad_fn=<NormBackward1>) at iteration 0
Loss before: tensor(100.4572, grad_fn=<NormBackward1>)
Lo

torch.Size([100, 1])
torch.Size([100, 1])


KeyboardInterrupt: 

### Time at different data sizes
### Here we run 1000 inner loops and 10 in the outer
#### For X = 3 x 17  
##### It took: 17.8 seconds


#### For X = 6 x 17  
##### It took: 28.08 seconds


#### For X = 12 x 17
#### It took: 48 seconds




In [274]:
def furthestSum(X):
    idx = np.random.choice(range(0,N))
    x_j = X[:,idx]
    max_vals = [0]
    max_vals = max_vals * K

    b_j = np.empty((K,1))
    b_j_norm = np.empty((K,1))

    for i in range(N):
        val = 0
        for j in range(M):
            val += np.abs(X[j, i] - x_j[j])
        max_idxs = [j for j, v in enumerate(max_vals) if val > v]

        if len(max_idxs) > 0:
            max_vals[max_idxs[0]] = val
            b_j = X[:, i]
            
    return b_j

In [291]:
# Choose a random point for initialization
idx = int(np.random.choice(range(0,N)))
x_j = X[:,idx]

j_news = list()
j_news.append(idx)

excluded = [idx]

# Loop over the K archetypes
for n in range(K):
    best_val = 0
    best_idx = 0
    # Loop over all unseen samples
    for i in range(N-len(excluded)):
        if i not in excluded:
            val = 0
            # sum over each element for each point
            for ele in j_news:
                for j in range(M):
                    val += np.abs(X[j,i] - X[j , ele])
            if val > best_val:
                best_val = val
                best_idx = i
    
    # 
    j_news.append(best_idx)
    excluded.append(best_idx)
    # Remove the random initialization
    if n == 0:
        j_news.pop(0)
        excluded.pop(0)
        
    
j_news

[83, 0, 14, 73, 47]